1단계: 데이터 로딩 및 초기 확인
주요 작업
PySpark DataFrame 생성
CSV 파일을 PySpark의 read.csv() 메서드를 이용해 읽어옵니다.

header=True: 첫 번째 줄을 헤더로 사용합니다.
inferSchema=True: 컬럼 데이터 타입을 자동으로 추론합니다.
데이터 확인

df.show(truncate=False): 데이터 샘플 확인.
df.printSchema(): 데이터의 구조(컬럼 이름, 타입 등)를 확인.


In [1]:
from pyspark.sql import SparkSession

# Spark 세션 초기화
spark = SparkSession.builder.appName("Ironman Data Analysis_241219_02").getOrCreate()

# CSV 다시 불러오기
file_path = "file:///home/lab12/src/data/ironman_wc_2022.csv"  # 정확한 경로 입력
df = spark.read.csv(file_path, header=True, inferSchema=True)

# 데이터 확인
df.show(truncate=False)
df.printSchema()

24/12/20 11:48:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+---+--------------------+--------------+------+----+--------+------------+------------+---------+---------+---------+---------+--------+--------+-------------+
|bib|name                |country       |gender|div |div_rank|overall_time|overall_rank|swim_time|swim_rank|bike_time|bike_rank|run_time|run_rank|finish_status|
+---+--------------------+--------------+------+----+--------+------------+------------+---------+---------+---------+---------+--------+--------+-------------+
|8  |Gustav Iden         |Norway        |Male  |MPRO|1       |7:40:24     |1           |48:23:00 |10       |4:11:06  |6        |2:36:15 |1       |Finisher     |
|15 |Sam Laidlow         |France        |Male  |MPRO|2       |7:42:24     |2           |48:16:00 |2        |4:04:36  |1        |2:44:40 |5       |Finisher     |
|1  |Kristian Blummenfelt|Norway        |Male  |MPRO|3       |7:43:23     |3           |48:20:00 |5        |4:11:16  |8        |2:39:21 |2       |Finisher     |
|23 |Max Neumann         |Australi

2단계: 시간 데이터 처리

대회 기록 데이터를 문자열 형식(hh:mm:ss)에서 초 단위 데이터로 변환합니다.
변환된 데이터를 새로운 컬럼으로 추가해, 이후 분석과 모델 학습에 활용할 준비를 합니다.

시간 데이터를 초 단위로 변환

time_to_seconds 함수는 문자열 시간 데이터를 분리하여 초 단위로 계산합니다.
split(time_str_col, ":"): hh:mm:ss 문자열을 : 기준으로 나눕니다.
[0], [1], [2]: 각각 시, 분, 초를 나타냅니다.
.cast("int"): 문자열 값을 정수로 변환.

새로운 컬럼 추가

withColumn()을 사용하여 기존 시간 데이터를 변환한 결과를 새로운 컬럼으로 추가.
swim_seconds: 수영 기록(초 단위).
bike_seconds: 자전거 기록(초 단위).
run_seconds: 달리기 기록(초 단위).
overall_seconds: 총합 기록(초 단위).


컷오프 기준 설정

수영 컷오프: 수영 기록이 2시간 20분(8400초)을 초과하면 DNF.
수영 + 자전거 컷오프: 수영 + 자전거 기록 합이 10시간 30분(37800초)을 초과하면 DNF.
총합 컷오프: 전체 기록이 17시간(61200초)을 초과하면 DNF.

조건문 사용 (when)

when 함수는 조건을 만족하면 1(DNF)을, 그렇지 않으면 0(완주)을 반환합니다.
여러 조건을 |(OR)로 연결하여, 한 가지라도 초과하면 1로 표시.
새로운 컬럼 추가 (withColumn)

DNF 컬럼은 각 참가자의 컷오프 상태를 나타냅니다:
1: 컷오프 초과 (DNF).
0: 컷오프 이내 (완주).

In [11]:
from pyspark.sql.functions import when, col, split

# 1. 시간 데이터를 초 단위로 변환하는 함수 정의
def time_to_seconds(time_str_col):
    """
    문자열 형태의 시간 데이터를 초 단위로 변환
    """
    return (split(time_str_col, ":")[0].cast("int") * 3600 +
            split(time_str_col, ":")[1].cast("int") * 60 +
            split(time_str_col, ":")[2].cast("int"))

# 2. 초 단위 컬럼 생성
df = df.withColumn("swim_seconds", time_to_seconds(col("swim_time"))) \
       .withColumn("bike_seconds", time_to_seconds(col("bike_time"))) \
       .withColumn("run_seconds", time_to_seconds(col("run_time"))) \
       .withColumn("overall_seconds", time_to_seconds(col("overall_time")))

# 3. 컷오프 기준에 따른 DNF 컬럼 생성
df = df.withColumn(
    "DNF",
    when((col("swim_seconds") > 2 * 3600 + 20 * 60) |  # 수영 컷오프
         (col("swim_seconds") + col("bike_seconds") > 10 * 3600 + 30 * 60) |  # 수영 + 사이클 컷오프
         (col("overall_seconds") > 17 * 3600), 1).otherwise(0)  # 전체 컷오프
)

# 4. 결과 확인
df.select("swim_seconds", "bike_seconds", "run_seconds", "overall_seconds", "DNF").show()

+------------+------------+-----------+---------------+---+
|swim_seconds|bike_seconds|run_seconds|overall_seconds|DNF|
+------------+------------+-----------+---------------+---+
|      174180|       15066|       9375|          27624|  1|
|      173760|       14676|       9880|          27744|  1|
|      174000|       15076|       9561|          27803|  1|
|      174300|       15090|       9614|          27884|  1|
|      190500|       15071|       9926|          28445|  1|
|      190680|       14951|      10125|          28540|  1|
|      190440|       14945|      10168|          28552|  1|
|      179340|       15218|      10091|          28598|  1|
|      179400|       15314|       9960|          28618|  1|
|      178920|       15712|       9719|          28700|  1|
|      190260|       14944|      10467|          28851|  1|
|      173700|       15478|      10229|          28913|  1|
|      174180|       15210|      10563|          28978|  1|
|      174360|       15779|      10023| 

DNS/DQ 상태 제거

DNS: "Did Not Start" (참가하지 않음).
DQ: "Disqualified" (실격).
이 두 상태는 대회 규칙상 기록이 없거나 무효 처리된 데이터이므로 제거합니다.
결측값 제거

dropna()를 사용해 특정 컬럼(swim_seconds, bike_seconds, run_seconds, overall_seconds)에 결측값이 있으면 해당 행을 제거합니다.
모든 기록이 없는 데이터는 분석과 모델 학습에 방해가 되기 때문입니다.
데이터 타입 변환

숫자 연산이 정확하게 이루어지도록 기록 데이터를 double(실수형)로 변환합니다.
이는 이후 학습 모델에서 데이터 타입 불일치로 인한 오류를 방지합니다.

In [12]:
df = df.filter((col("finish_status") != "DNS") & (col("finish_status") != "DQ"))

In [13]:
df = df.dropna(subset=["swim_seconds", "bike_seconds", "run_seconds", "overall_seconds"])

In [14]:
df = df.withColumn("swim_seconds", col("swim_seconds").cast("double")) \
       .withColumn("bike_seconds", col("bike_seconds").cast("double")) \
       .withColumn("run_seconds", col("run_seconds").cast("double")) \
       .withColumn("overall_seconds", col("overall_seconds").cast("double"))

In [15]:
df.groupBy("DNF").count().show()

+---+-----+
|DNF|count|
+---+-----+
|  1|  345|
|  0| 2031|
+---+-----+



In [16]:
df_finishers = df.filter(col("DNF") == 0)
df_finishers.select("DNF").groupBy("DNF").count().show()

+---+-----+
|DNF|count|
+---+-----+
|  0| 2031|
+---+-----+



In [17]:
from pyspark.ml.feature import StringIndexer

# gender 컬럼 인코딩
indexer = StringIndexer(inputCol="gender", outputCol="gender_encoded")
df_finishers = indexer.fit(df_finishers).transform(df_finishers)

# 결과 확인
df_finishers.select("gender", "gender_encoded").show(5)

+------+--------------+
|gender|gender_encoded|
+------+--------------+
|  Male|           0.0|
|  Male|           0.0|
|  Male|           0.0|
|  Male|           0.0|
|  Male|           0.0|
+------+--------------+
only showing top 5 rows



In [19]:
from pyspark.sql.functions import when, col

# div 컬럼을 기반으로 age_group 생성
df_finishers = df_finishers.withColumn(
    "age_group",
    when(col("div").startswith("M18-24"), 18)
    .when(col("div").startswith("M25-29"), 25)
    .when(col("div").startswith("M30-34"), 30)
    .when(col("div").startswith("M35-39"), 35)
    .when(col("div").startswith("M40-44"), 40)
    .when(col("div").startswith("M45-49"), 45)
    .when(col("div").startswith("M50-54"), 50)
    .when(col("div").startswith("M55-59"), 55)
    .when(col("div").startswith("MPRO"), 0)  # 프로 선수
    .otherwise(None)  # 나머지 경우 처리
)

# 결과 확인
df_finishers.select("div", "age_group").distinct().show()

+------+---------+
|   div|age_group|
+------+---------+
|M30-34|       30|
|M25-29|       25|
|M18-24|       18|
|M35-39|       35|
|M40-44|       40|
|M55-59|       55|
|M45-49|       45|
|M50-54|       50|
+------+---------+



전체 참가자 수 계산

df.count(): 데이터셋에 포함된 전체 참가자 수를 계산합니다.
이를 바탕으로 각 그룹의 기준(Top 10%, Top 25%, 등)을 설정합니다.
순위 그룹 기준 설정

Top 10%: 전체 참가자 중 상위 10%.
Top 25%: 상위 10% 이후 ~ 상위 25% 이내.
Top 50%: 상위 25% 이후 ~ 상위 50% 이내.
Bottom 50%: 하위 50%.
rank_range 컬럼 생성

when 조건문을 사용해 각 참가자의 overall_rank를 기준으로 rank_range 컬럼에 그룹 값을 할당합니다:
예: overall_rank가 100명 중 5위라면 Top 10%.

In [20]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import when, col

# 1. 순위 그룹 라벨링
total_participants = df_finishers.count()
top_10 = total_participants * 0.1
top_25 = total_participants * 0.25
top_50 = total_participants * 0.5

df_finishers = df_finishers.withColumn(
    "rank_range",
    when(col("overall_rank") <= top_10, "Top 10%")
    .when((col("overall_rank") > top_10) & (col("overall_rank") <= top_25), "Top 25%")
    .when((col("overall_rank") > top_25) & (col("overall_rank") <= top_50), "Top 50%")
    .otherwise("Bottom 50%")
)

# 2. 피처 벡터 생성
assembler = VectorAssembler(
    inputCols=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"],
    outputCol="features"
)

df_final = assembler.transform(df_finishers).select("features", "rank_range")

# 결과 확인
df_final.show(5, truncate=False)

+---------------------------------+----------+
|features                         |rank_range|
+---------------------------------+----------+
|[0.0,30.0,3655.0,16953.0,11146.0]|Top 10%   |
|[0.0,30.0,3983.0,17318.0,10560.0]|Top 10%   |
|[0.0,30.0,3755.0,17022.0,11094.0]|Top 10%   |
|[0.0,35.0,4042.0,16196.0,11461.0]|Top 10%   |
|[0.0,30.0,3955.0,17532.0,10395.0]|Top 10%   |
+---------------------------------+----------+
only showing top 5 rows



In [21]:
# rank_range를 숫자형 rank_range_index로 변환
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="rank_range", outputCol="rank_range_index")
df_final = indexer.fit(df_final).transform(df_final)

# 결과 확인
df_final.select("rank_range", "rank_range_index").distinct().show()

+----------+----------------+
|rank_range|rank_range_index|
+----------+----------------+
|   Top 10%|             3.0|
|   Top 25%|             2.0|
|Bottom 50%|             0.0|
|   Top 50%|             1.0|
+----------+----------------+



randomSplit()을 사용해 데이터를 학습용(80%)과 테스트용(20%)으로 나눕니다:
학습 데이터(train_data): 모델 학습에 사용.
테스트 데이터(test_data): 학습된 모델의 성능 평가에 사용.
seed: 데이터를 분할할 때 결과를 재현 가능하게 하는 고정값.

In [22]:
# 데이터 분할
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)

# 데이터 크기 확인
print("훈련 데이터 크기:", train_data.count())
print("테스트 데이터 크기:", test_data.count())

# 데이터 샘플 확인
train_data.show(5, truncate=False)
test_data.show(5, truncate=False)

훈련 데이터 크기: 1671
테스트 데이터 크기: 360
+---------------------------------+----------+----------------+
|features                         |rank_range|rank_range_index|
+---------------------------------+----------+----------------+
|[0.0,18.0,3653.0,20368.0,12874.0]|Top 50%   |1.0             |
|[0.0,18.0,3693.0,19738.0,17137.0]|Bottom 50%|0.0             |
|[0.0,18.0,3722.0,19532.0,17446.0]|Bottom 50%|0.0             |
|[0.0,18.0,3729.0,18523.0,14229.0]|Top 50%   |1.0             |
|[0.0,18.0,3732.0,18024.0,16481.0]|Bottom 50%|0.0             |
+---------------------------------+----------+----------------+
only showing top 5 rows

+---------------------------------+----------+----------------+
|features                         |rank_range|rank_range_index|
+---------------------------------+----------+----------------+
|[0.0,18.0,3717.0,19283.0,13253.0]|Top 50%   |1.0             |
|[0.0,18.0,3776.0,18633.0,12743.0]|Top 50%   |1.0             |
|[0.0,18.0,3855.0,19772.0,14458.0]|Bottom 50%|0

In [ ]:
Random Forest 모델 생성

RandomForestClassifier: 분류 모델로, 여러 개의 의사결정 트리를 앙상블 방식으로 사용.
labelCol: 모델의 레이블(정답) 컬럼 (rank_range_index).
featuresCol: 모델의 입력값(특성 벡터) 컬럼 (features).
numTrees: 생성할 트리의 개수 (여기서는 50개).

모델 학습

rf.fit(train_data): 학습 데이터를 사용해 Random Forest 모델을 학습.

테스트 데이터로 예측

rf_model.transform(test_data): 학습된 모델로 테스트 데이터를 예측.
prediction: 모델이 예측한 순위 그룹.
probability: 각 순위 그룹에 속할 확률.

In [23]:
from pyspark.ml.classification import RandomForestClassifier

# Random Forest 모델 생성 및 학습
rf = RandomForestClassifier(labelCol="rank_range_index", featuresCol="features", numTrees=50)
rf_model = rf.fit(train_data)

# 테스트 데이터로 예측
predictions = rf_model.transform(test_data)

# 결과 확인
predictions.select("features", "rank_range_index", "prediction", "probability").show(10, truncate=False)

+---------------------------------+----------------+----------+----------------------------------------------------------------------------------+
|features                         |rank_range_index|prediction|probability                                                                       |
+---------------------------------+----------------+----------+----------------------------------------------------------------------------------+
|[0.0,18.0,3717.0,19283.0,13253.0]|1.0             |1.0       |[0.15239751076467112,0.814753365129092,0.03277894866764049,7.017543859649125E-5]  |
|[0.0,18.0,3776.0,18633.0,12743.0]|1.0             |1.0       |[0.05188127046916252,0.598255425232573,0.3097798372675162,0.0400834670307482]     |
|[0.0,18.0,3855.0,19772.0,14458.0]|0.0             |0.0       |[0.941644413078116,0.058355586921883945,0.0,0.0]                                  |
|[0.0,18.0,3943.0,18864.0,14091.0]|1.0             |1.0       |[0.407518233720109,0.5533031052250287,0.039019596727377

예측 수행

rf_model.transform(test_data)를 통해 테스트 데이터에서 예측을 수행합니다:
prediction: 모델이 예측한 순위 그룹 (0: Bottom 50%, 1: Top 50%, 2: Top 25%, 3: Top 10%).
probability: 각 순위 그룹에 속할 확률 분포.

결과 출력

select()로 관심 있는 컬럼을 선택해 결과를 확인합니다:
features: 모델에 입력된 특성 벡터.
rank_range_index: 실제 레이블(정답).
prediction: 모델의 예측값.
probability: 각 클래스에 속할 확률 벡터.

In [ ]:
#테스트 결과 확인

모델 정확도와 F1-스코어
모델 정확도: 0.95
전체 예측 중 95%가 정답이었다는 의미입니다.
매우 높은 정확도를 보여주는 결과입니다.
모델 F1-스코어: 0.95
Precision(정밀도)과 Recall(재현율)을 조화롭게 고려한 지표로, 모델이 클래스 불균형 데이터에서도 우수한 성능을 보였음을 나타냅니다.
정확도와 F1-스코어가 동일하거나 비슷하다면, 모델이 전반적으로 균형 잡힌 성능을 가지고 있음을 의미합니다.



In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 평가 지표: 정확도
evaluator = MulticlassClassificationEvaluator(
    labelCol="rank_range_index",
    predictionCol="prediction",
    metricName="accuracy"
)

accuracy = evaluator.evaluate(predictions)
print(f"모델 정확도: {accuracy:.2f}")


모델 정확도: 0.95


In [25]:
# 평가 지표: F1-스코어
evaluator_f1 = MulticlassClassificationEvaluator(
    labelCol="rank_range_index",
    predictionCol="prediction",
    metricName="f1"
)

f1_score = evaluator_f1.evaluate(predictions)
print(f"모델 F1-스코어: {f1_score:.2f}")


모델 F1-스코어: 0.95


rank_range_index: 실제 정답 레이블.
prediction: 모델이 예측한 값.
count: 해당 레이블과 예측값의 조합에 속하는 데이터의 개수.
예시 분석:

1.0 → 1.0 (76)
실제 순위 그룹이 Top 50%(1.0)인 참가자 76명을 정확히 예측했습니다.
3.0 → 2.0 (4)
실제 Top 10%(3.0)인 참가자 4명을 Top 25%(2.0)로 잘못 예측했습니다.
0.0 → 0.0 (234)
실제 Bottom 50%(0.0)인 참가자 234명을 정확히 예측했습니다.
1.0 → 0.0 (4)
실제 Top 50%(1.0)인 참가자 4명을 Bottom 50%(0.0)로 잘못 예측했습니다.

In [26]:
# 예측 및 실제값 카운트
predictions.groupBy("rank_range_index", "prediction").count().show()


+----------------+----------+-----+
|rank_range_index|prediction|count|
+----------------+----------+-----+
|             1.0|       1.0|   76|
|             3.0|       2.0|    4|
|             0.0|       1.0|    4|
|             1.0|       0.0|    4|
|             2.0|       2.0|   26|
|             2.0|       1.0|    3|
|             1.0|       2.0|    2|
|             0.0|       0.0|  234|
|             3.0|       3.0|    7|
+----------------+----------+-----+



In [32]:
# 특성 중요도 확인
feature_importance = rf_model.featureImportances
features = ["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"]

# 결과 출력
for feature, importance in zip(features, feature_importance):
    print(f"{feature}: {importance:.2f}")


gender_encoded: 0.00
age_group: 0.01
swim_seconds: 0.07
bike_seconds: 0.36
run_seconds: 0.57


사용자 입력 받기

성별, 나이, 수영, 자전거, 달리기 기록을 사용자로부터 입력받습니다.
입력받은 기록을 Ironman 대회의 거리 기준으로 변환합니다.
컷오프 기준 확인

수영 + 자전거 기록, 전체 기록을 컷오프 기준과 비교합니다.
기준을 초과하면 완주 불가능 메시지를 출력합니다.
모델 예측

입력 데이터를 특성 벡터로 변환하고, 학습된 모델에 전달합니다.
모델은 순위 그룹을 예측하고 확률 분포를 반환합니다.
예상 기록과 개선 방향 제공

모델 예측 결과를 기반으로 예상 순위 그룹을 출력합니다.
예상 기록(수영, 자전거, 달리기)과 상위 10% 진입을 위한 목표 기록을 제시합니다.

In [33]:
# 필요한 모듈 임포트
from pyspark.ml.classification import RandomForestClassifier

# 시간 데이터를 초 단위로 변환하는 함수
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(":"))
    return h * 3600 + m * 60 + s

# 초를 hh:mm:ss 형식으로 변환하는 함수
def seconds_to_time(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02}:{m:02}:{s:02}"

# 클래스 매핑
class_mapping = {
    0.0: "Bottom 50%",
    1.0: "Top 50%",
    2.0: "Top 25%",
    3.0: "Top 10%"
}

# 예상 기록 계산 함수
def calculate_average_times(predicted_group):
    avg_times = {
        "Bottom 50%": {"swim": 1.5 * 3600, "bike": 6.5 * 3600, "run": 4.5 * 3600},
        "Top 50%": {"swim": 1.4 * 3600, "bike": 6.0 * 3600, "run": 4.0 * 3600},
        "Top 25%": {"swim": 1.3 * 3600, "bike": 5.5 * 3600, "run": 3.5 * 3600},
        "Top 10%": {"swim": 1.2 * 3600, "bike": 5.0 * 3600, "run": 3.0 * 3600},
    }
    return avg_times[predicted_group]

# 사용자 입력 데이터 받기
print("자신의 정보를 입력해주세요:")
gender = input("성별 (남성/여성): ")
age = int(input("나이: "))
swim_time = input("수영 기록 (hh:mm:ss, 1.5km 기준): ")
bike_time = input("자전거 기록 (hh:mm:ss, 40km 기준): ")
run_time = input("달리기 기록 (hh:mm:ss, 10km 기준): ")

# 성별 인코딩 및 시간 변환
gender_encoded = 1.0 if gender == "남성" else 0.0
swim_seconds = time_to_seconds(swim_time) * (3.8 / 1.5)  # 수영 3.8km로 변환
bike_seconds = time_to_seconds(bike_time) * (180 / 40)  # 자전거 180km로 변환
run_seconds = time_to_seconds(run_time) * (42.2 / 10)  # 달리기 42.2km로 변환

# 컷오프 기준
SWIM_BIKE_CUTOFF = 10 * 3600 + 30 * 60  # 수영 + 자전거: 10시간 30분
TOTAL_CUTOFF = 17 * 3600  # 전체 기록: 17시간

# 입력 데이터 검증
total_swim_bike = swim_seconds + bike_seconds
total_time = total_swim_bike + run_seconds

if total_swim_bike > SWIM_BIKE_CUTOFF or total_time > TOTAL_CUTOFF:
    print("\n완주 불가능: 입력된 기록이 대회 컷오프 기준을 초과했습니다.")
    print(f"    - 수영 + 자전거: {seconds_to_time(total_swim_bike)} (컷오프: {seconds_to_time(SWIM_BIKE_CUTOFF)})")
    print(f"    - 전체 시간: {seconds_to_time(total_time)} (컷오프: {seconds_to_time(TOTAL_CUTOFF)})")
else:
    # 입력 데이터 생성
    input_data = [[gender_encoded, age, swim_seconds, bike_seconds, run_seconds]]
    input_df = spark.createDataFrame(input_data, schema=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"])
    input_features = assembler.transform(input_df)

    # 모델 예측
    predictions = rf_model.transform(input_features)
    prediction = predictions.select("prediction", "probability").collect()[0]
    predicted_rank = class_mapping[prediction["prediction"]]
    probabilities = prediction["probability"]

    # 예상 기록 계산
    average_times = calculate_average_times(predicted_rank)

    # 결과 출력
    print("\n모델 예측 결과:")
    print(f"- 예상 종합 순위 그룹: {predicted_rank}")
    #print(f"- 예상 부문 순위 그룹 (성별: {gender}, 나이 그룹: {age // 10 * 10}대): {predicted_rank}")
    # print(f"- 예상 부문 순위 그룹 (성별: {gender}, 나이 그룹: {age // 10 * 10}대): {predicted_rank}")
    # 위 주석 처리된 출력은 모델 설계상 잘못된 정보일 수 있으므로 비활성화

    print("\n예상 기록:")
    print(f"    - 수영: {seconds_to_time(average_times['swim'])}")
    print(f"    - 자전거: {seconds_to_time(average_times['bike'])}")
    print(f"    - 달리기: {seconds_to_time(average_times['run'])}")

    print("\n종목별 개선 방향:")
    print("    - 수영(3.8km): 상위 10%에 진입하려면 1시간 3분 58초 이하로 줄여야 합니다.")
    print("    - 자전거(180km): 상위 10%에 진입하려면 4시간 48분 07초 이하로 줄여야 합니다.")
    print("    - 달리기(42.2km): 상위 10%에 진입하려면 3시간 15분 25초 이하로 줄여야 합니다.")



자신의 정보를 입력해주세요:
성별 (남성/여성): 남성
나이: 33
수영 기록 (hh:mm:ss, 1.5km 기준): 02:00:00
자전거 기록 (hh:mm:ss, 40km 기준): 02:00:00
달리기 기록 (hh:mm:ss, 10km 기준): 02:00:00

완주 불가능: 입력된 기록이 대회 컷오프 기준을 초과했습니다.
    - 수영 + 자전거: 14:04:00 (컷오프: 10:30:00)
    - 전체 시간: 22:30:24 (컷오프: 17:00:00)


------- 각 부문별 평균 기록을 계산해보자

In [28]:
# 데이터프레임 스키마 확인
df_final.printSchema()

# 컬럼 샘플 확인
df_final.show(5, truncate=False)


root
 |-- features: vector (nullable = true)
 |-- rank_range: string (nullable = false)
 |-- rank_range_index: double (nullable = false)

+---------------------------------+----------+----------------+
|features                         |rank_range|rank_range_index|
+---------------------------------+----------+----------------+
|[0.0,30.0,3655.0,16953.0,11146.0]|Top 10%   |3.0             |
|[0.0,30.0,3983.0,17318.0,10560.0]|Top 10%   |3.0             |
|[0.0,30.0,3755.0,17022.0,11094.0]|Top 10%   |3.0             |
|[0.0,35.0,4042.0,16196.0,11461.0]|Top 10%   |3.0             |
|[0.0,30.0,3955.0,17532.0,10395.0]|Top 10%   |3.0             |
+---------------------------------+----------+----------------+
only showing top 5 rows



In [29]:
from pyspark.ml.functions import vector_to_array

# 벡터 데이터를 배열로 변환
df_final = df_final.withColumn("feature_array", vector_to_array("features"))

# 배열에서 각 요소를 개별 컬럼으로 분리
df_final = df_final.withColumn("gender_encoded", col("feature_array")[0]) \
                   .withColumn("age_group", col("feature_array")[1]) \
                   .withColumn("swim_seconds", col("feature_array")[2]) \
                   .withColumn("bike_seconds", col("feature_array")[3]) \
                   .withColumn("run_seconds", col("feature_array")[4])

# 결과 확인
df_final.show(5, truncate=False)



+---------------------------------+----------+----------------+-------------------------------------+--------------+---------+------------+------------+-----------+
|features                         |rank_range|rank_range_index|feature_array                        |gender_encoded|age_group|swim_seconds|bike_seconds|run_seconds|
+---------------------------------+----------+----------------+-------------------------------------+--------------+---------+------------+------------+-----------+
|[0.0,30.0,3655.0,16953.0,11146.0]|Top 10%   |3.0             |[0.0, 30.0, 3655.0, 16953.0, 11146.0]|0.0           |30.0     |3655.0      |16953.0     |11146.0    |
|[0.0,30.0,3983.0,17318.0,10560.0]|Top 10%   |3.0             |[0.0, 30.0, 3983.0, 17318.0, 10560.0]|0.0           |30.0     |3983.0      |17318.0     |10560.0    |
|[0.0,30.0,3755.0,17022.0,11094.0]|Top 10%   |3.0             |[0.0, 30.0, 3755.0, 17022.0, 11094.0]|0.0           |30.0     |3755.0      |17022.0     |11094.0    |
|[0.0,35.0

In [30]:
from pyspark.sql.functions import avg

# Top 10% 그룹 데이터 추출
top10_group = df_final.filter(df_final["rank_range_index"] == 3.0)

# Top 10% 그룹의 평균 기록 계산
top10_avg = top10_group.agg(
    avg("swim_seconds").alias("avg_swim_seconds"),
    avg("bike_seconds").alias("avg_bike_seconds"),
    avg("run_seconds").alias("avg_run_seconds")
).toPandas()

# 초를 hh:mm:ss 형식으로 변환
def seconds_to_time(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02}:{m:02}:{s:02}"

# 결과 변환
top10_swim = seconds_to_time(top10_avg["avg_swim_seconds"][0])
top10_bike = seconds_to_time(top10_avg["avg_bike_seconds"][0])
top10_run = seconds_to_time(top10_avg["avg_run_seconds"][0])

print(f"Top 10% 그룹 평균 기록:")
print(f" - 수영 (3.8km): {top10_swim}")
print(f" - 자전거 (180km): {top10_bike}")
print(f" - 달리기 (42.2km): {top10_run}")
from pyspark.sql.functions import avg
from pyspark.ml.functions import vector_to_array

# 벡터 데이터를 배열로 변환
df_final = df_final.withColumn("feature_array", vector_to_array("features"))

# 배열에서 각 요소를 개별 컬럼으로 분리
df_final = df_final.withColumn("gender_encoded", col("feature_array")[0]) \
                   .withColumn("age_group", col("feature_array")[1]) \
                   .withColumn("swim_seconds", col("feature_array")[2]) \
                   .withColumn("bike_seconds", col("feature_array")[3]) \
                   .withColumn("run_seconds", col("feature_array")[4])

# 그룹별 평균값 계산
group_avg = df_final.groupBy("rank_range_index").agg(
    avg("swim_seconds").alias("avg_swim_seconds"),
    avg("bike_seconds").alias("avg_bike_seconds"),
    avg("run_seconds").alias("avg_run_seconds")
).toPandas()

# 초를 hh:mm:ss 형식으로 변환하는 함수
def seconds_to_time(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02}:{m:02}:{s:02}"

# 결과 변환 및 출력
print("순위 그룹별 평균 기록:")
rank_mapping = {3.0: "Top 10%", 2.0: "Top 25%", 1.0: "Top 50%", 0.0: "Bottom 50%"}
for _, row in group_avg.iterrows():
    rank = rank_mapping[row["rank_range_index"]]
    swim_time = seconds_to_time(row["avg_swim_seconds"])
    bike_time = seconds_to_time(row["avg_bike_seconds"])
    run_time = seconds_to_time(row["avg_run_seconds"])
    print(f"\n{rank} 그룹:")
    print(f" - 수영 (3.8km): {swim_time}")
    print(f" - 자전거 (180km): {bike_time}")
    print(f" - 달리기 (42.2km): {run_time}")




Top 10% 그룹 평균 기록:
 - 수영 (3.8km): 01:03:58
 - 자전거 (180km): 04:48:07
 - 달리기 (42.2km): 03:15:25


순위 그룹별 평균 기록:

Bottom 50% 그룹:
 - 수영 (3.8km): 01:16:55
 - 자전거 (180km): 05:46:42
 - 달리기 (42.2km): 04:41:09

Top 50% 그룹:
 - 수영 (3.8km): 01:08:23
 - 자전거 (180km): 05:07:53
 - 달리기 (42.2km): 03:44:57

Top 10% 그룹:
 - 수영 (3.8km): 01:03:58
 - 자전거 (180km): 04:48:07
 - 달리기 (42.2km): 03:15:25

Top 25% 그룹:
 - 수영 (3.8km): 01:06:09
 - 자전거 (180km): 04:56:43
 - 달리기 (42.2km): 03:29:18


In [34]:
spark.stop()

24/12/20 14:43:18 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-04002838-ebf5-45ab-8efc-8bca5d12858a/pyspark-915e155c-3656-4ee5-96ce-3e05ad389aae. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-04002838-ebf5-45ab-8efc-8bca5d12858a/pyspark-915e155c-3656-4ee5-96ce-3e05ad389aae
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1141)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4$adapted(ShutdownHookManager.scala:62)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreac